In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from scipy import stats
from scipy.stats import norm, skew
from sklearn import preprocessing
#preprocessing :
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler , StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split


from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import  Normalizer , scale
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV , KFold , cross_val_score
from sklearn.metrics import r2_score

In [ ]:
df = pd.read_csv('train.csv')
df.drop('Id',axis=1,inplace=True)

In [ ]:
df.loc[df[(df['z']==0)].index,'z']=df['z'].median()
df.loc[df[df['x']==0].index,'x']=df['x'].median()
df.loc[df[(df['y']==0)].index,'y']=df['y'].median() 

In [ ]:
df.describe()

,carat,depth,table,price,x,y,z
count,43152.000000,43152.000000,43152.000000,43152.000000,43152.000000,43152.000000,43152.000000
mean,0.797855,61.747177,57.458347,3929.491912,5.732228,5.735547,3.539958
std,0.473594,1.435454,2.233904,3985.527795,1.119580,1.147481,0.704745
min,0.200000,43.000000,43.000000,326.000000,3.730000,3.680000,1.070000
25%,0.400000,61.000000,56.000000,947.750000,4.710000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5312.000000,6.540000,6.540000,4.040000
max,5.010000,79.000000,95.000000,18823.000000,10.740000,58.900000,31.800000


In [ ]:
len(df.query("carat>3"))

21

In [ ]:
#df.loc[df[df['carat']>3].index,'carat']=df['carat'].median()

In [ ]:
df['volume'] = df['x']*df['y']*df['z']
df.drop(['x','y','z'], axis=1, inplace= True)
df.head()

,carat,cut,color,clarity,depth,table,price,volume
0,1.06,Ideal,I,SI2,61.8,57.0,4270,176.483340
1,1.51,Premium,G,VVS2,60.9,58.0,15164,246.965796
2,0.32,Ideal,F,VS2,61.3,56.0,828,52.943373
3,0.53,Ideal,G,VS2,61.2,56.0,1577,86.422842
4,0.70,Premium,H,VVS2,61.0,57.0,2596,115.315200


In [ ]:
X = df.drop(['price'], axis=1)
y = df['price']

X_train, X_vali, y_train, y_vali = train_test_split(X,y,test_size=0.2, random_state=42)

In [ ]:
label_cut = preprocessing.LabelEncoder()
label_color = preprocessing.LabelEncoder()
label_clarity = preprocessing.LabelEncoder()


X_train['cut'] = label_cut.fit_transform(X_train['cut'])
X_train['color'] = label_color.fit_transform(X_train['color'])
X_train['clarity'] = label_clarity.fit_transform(X_train['clarity'])

In [ ]:
X_vali['cut'] = label_cut.transform(X_vali['cut'])
X_vali['color'] = label_color.transform(X_vali['color'])
X_vali['clarity'] = label_clarity.transform(X_vali['clarity'])

In [ ]:
sc = MinMaxScaler()
X_train = sc.fit_transform(X_train)
X_vali = sc.transform(X_vali)

In [ ]:
df = pd.read_csv('test.csv')
Id = df['Id']
df.drop('Id',axis=1,inplace=True)
#*************
df.loc[df[(df['z']==0)].index,'z']=df['z'].median()
df.loc[df[df['x']==0].index,'x']=df['x'].median()
df.loc[df[(df['y']==0)].index,'y']=df['y'].median() 
#****************
#df.loc[df[df['carat']>3].index,'carat']=df['carat'].median()
#**************
df['volume'] = df['x']*df['y']*df['z']
df.drop(['x','y','z'], axis=1, inplace= True)
#********************
df['cut'] = label_cut.transform(df['cut'])
df['color'] = label_color.transform(df['color'])
df['clarity'] = label_clarity.transform(df['clarity'])
#*************
test = sc.transform(df)

In [ ]:
#no_of_test=[100]
#params_dict={'n_estimators':no_of_test,'n_jobs':[-1],'max_features':["auto",'sqrt','log2']}
#clf_rf=GridSearchCV(estimator=RandomForestRegressor(),param_grid=params_dict,scoring='r2')
#clf_rf.fit(X_train,y_train)
#print('Score : %.4f' % clf_rf.score(X_vali, y_vali))
#pred=clf_rf.predict(X_vali)
#r2 = r2_score(y_vali, pred)
#print('R2     : %0.2f ' % r2)


In [ ]:
#y_test_predict = clf_rf.predict(test)
#0.9810
#R2     : 0.98 

In [ ]:
#R = pd.DataFrame({'Id' : Id , 'price' : y_test_predict})
#R.to_csv("GridSearchCV-random-result.csv" , index=False) 

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_vali, label=y_vali)
test=xgb.DMatrix(test)

In [ ]:
params = {
    # Parameters that we are going to tune.
    'max_depth':10,
    "learning_rate" :0.1,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    "n_estimators" : 500,
    'objective':'reg:squarederror',
}

In [ ]:
params['eval_metric'] = "mae"
num_boost_round = 999

In [ ]:
model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)

[11:41:08] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[0]	Test-mae:3531.48130
[1]	Test-mae:3180.48638
[2]	Test-mae:2864.68942
[3]	Test-mae:2580.03888
[4]	Test-mae:2323.68229
[5]	Test-mae:2093.37592
[6]	Test-mae:1886.36239
[7]	Test-mae:1699.92522
[8]	Test-mae:1531.71312
[9]	Test-mae:1380.69379
[10]	Test-mae:1244.68698
[11]	Test-mae:1122.58181
[12]	Test-mae:1014.27092
[13]	Test-mae:917.87482
[14]	Test-mae:832.65636
[15]	Test-mae:757.06259
[16]	Test-mae:690.16512
[17]	Test-mae:631.19603
[18]	Test-mae:580.07490
[19]	Test-mae:535.44466
[20]	Test-mae:496.78749
[21]	Test-mae:462.99768
[22]	Test-mae:434.36768
[23]	Test-mae:409.62308
[24]	Test-mae:388.58993
[25]	Test-mae:370.37275
[26]	Test-mae:354.94072
[27]	Test-mae:341.97114
[28]	Test-mae:330.79413
[29]	Test-mae:321.17339
[30]	Test-mae:313.28258
[31]	Test-mae:306.56089
[32]	Test-mae:300.86308
[33]	Test-mae:296.14720
[34]	Test-mae:292.00106
[35]	Test-mae:288.56223
[36]	Test-mae:285.59005
[37]	Test-mae:282.8

In [ ]:
print("Best MAE: {:.2f} with {} rounds".format(
                 model.best_score,
                 model.best_iteration+1))


Best MAE: 267.12 with 93 rounds


In [ ]:
y_test_predict=model.predict(test)

In [ ]:
R = pd.DataFrame({'Id' : Id , 'price' : y_test_predict})
R.to_csv("Xgb-median-MinMaxScaler-result1.csv" , index=False) 